### Try to find a way to compute the synchrony of neurons accross time

In [105]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter


import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt
%matplotlib qt

import seaborn as sns 
import bk.load
import bk.compute
import bk.plot

import basefunction.anass_load as at

In [21]:
def binEdges2centers(BinEdges):
    #Vitor Code (Thanks)
    BinCenters = np.convolve(BinEdges,[.5,.5],'same')
    BinCenters = BinCenters[1::]

    return BinCenters

In [149]:
def binSpikes(neurons,binSize = 0.05,start = 0,stop = 0):
    if stop == 0:
        stop = np.max([neuron.as_units('s').index[-1] for neuron in neurons])

    bins = np.arange(start,stop,binSize)
    binned = []
    for neuron in neurons:
        hist,b = np.histogram(neuron.as_units('s').index,bins = bins)
        binned.append(hist)

    return np.array(binned), binEdges2centers(b)

def plotIntervals(intervals,col):
    for interval in intervals.as_units('s').values:
        plt.axvspan(interval[0],interval[1], facecolor=col, alpha=0.5)
        
# def synchrony(neurons,metadata,binSize,stru,sm = 201,method = 'corr',plot = True):
#     try: bk.load.session
#     except NameError: bk.load.session = None
    
# #     binSize = 0.1
#     winSize = int(1/binSize)
#     step = int(1/binSize)

    
#     print(len(neurons))
#     print(binSize)
    

#     binned,t = binSpikes(neurons,binSize)
    
#     stru_shank = {}
#     stru_mask  = {}
    
#     for s in stru:
#             stru_shank.update({s:at.get_structure_shanks(bk.load.session,s)})
#             if not np.any(stru_shank[s]):
#                 print('Could not compute ' + s + ' because is you don\'t have shank in it')
#                 continue
#             stru_mask.update({s:[i in stru_shank[s] for i in metadata[:,0]]})
    
#     if method == 'corr':
#         pbar = ProgressBar()
#         corr = np.empty((int(binned.shape[1]/step)+1,len(neurons),len(neurons)))   

#         for i,j in enumerate(pbar(range(0,binned.shape[1],step))):
#             corr[i] = np.corrcoef(binned[:,j:j+winSize])
#             corr[i][np.isnan(corr[i])] = 0
#         t = t[::step]
#         # corr = np.abs(corr)

#         stru_corr  = {}
#         stru_sync  = {}
            
#         for s in stru_mask.keys():
#             print('computing '+s)
#             stru_corr.update({s:corr[:,:,stru_mask[s]][:,stru_mask[s]]})
#             stru_sync.update({s:np.nanmean(stru_corr[s],axis = (1,2))})

#         stru_corr.update({'Hpc-BLA':corr[:,:,stru_mask['Hpc']][:,stru_mask['BLA']]})
#         stru_sync.update({'Hpc-BLA':np.nanmean(stru_corr['Hpc-BLA'],axis = (1,2))})

#         # for s,s2 in itertools.combinations(stru_mask.keys(),2):
#         #     print('computing '+s+'-'+s2)
#         #     stru_corr.update({s+'-'+s2:corr[:,:,stru_mask[s]][:,stru_mask[s2]]})
#         #     stru_sync.update({s+'-'+s2:np.nanmean(stru_corr[s+'-'+s2],axis = (1,2))})

#     elif method == 'coactivation':
#         #Binarize the binning.
#         #count number of cell active at each timebin
#         # nb of cell active / total number in da category for each time point. 
    
#         binarize_binned = np.where(binned<1,binned,1)
#         stru_sync  = {}
#         for s in stru_mask.keys():
#             print('computing '+s)
#             stru_sync.update({s:np.sum(binarize_binned[stru_mask[s],:],0)/binarize_binned.shape[0]})
            
#     elif method == 'firing':
#         stru_sync  = {}
#         for s in stru_mask.keys():
#             print('computing '+s)
#             stru_sync.update({s:np.mean(binned[stru_mask[s],:],0)/binSize})
#     else:    

#         print(method + ' Is an unknown method')
#         raise ValueError('Unknow Method')
    
    
#     if plot:
#         for s in stru_sync:
#             plt.plot(t,savgol_filter(stru_sync[s],sm,3))
#         plt.title('synchrony during ' + bk.session)
#         plt.legend([s for s in stru_sync.keys()])
#         plt.ylabel('r')
#         plt.xlabel('Time(s)')

#         Rem = nts.IntervalSet(scipy.io.loadmat('States.mat')['Rem'][:,0],scipy.io.loadmat('States.mat')['Rem'][:,1],time_units = 's')
#         plotIntervals(Rem,'darkorange')

#         sws = nts.IntervalSet(scipy.io.loadmat('States.mat')['sws'][:,0],scipy.io.loadmat('States.mat')['sws'][:,1],time_units = 's')
#         plotIntervals(sws,'grey')
    
#     return stru_sync
def synchrony(neurons,binSize,sm = 201,method = 'corr',plot = True):
    try: bk.load.session
    except NameError : bk.load.session = None
        
    winSize = int(500/binSize) 
    step = int(500/binSize)
    
    print(len(neurons),' neurons')
    
    binned,t = bk.compute.binSpikes(neurons,binSize)
    
    if method.lower() == 'corr':
        pbar = ProgressBar()
        
        corr = np.empty(int(binned.shape[1]/step)+1)
        for i,j in enumerate(pbar(range(0,binned.shape[1],step))):
            corr_mat = np.corrcoef(binned[:,j:j+winSize])
#             corr_mat[np.isnan(corr_mat)] = 0
            corr[i] = np.nanmean(corr_mat)
        t = t[::step]
        
        return t,corr
# def main(path):
#     neurons,metadata = bk.loadSpikeData(path)
#     synchrony = synchrony(neurons,metadata,['Hpc','BLA'])
    
#     return neurons,metadata

In [150]:
bk.load.current_session()
neurons,metadata = bk.load.loadSpikeData(bk.load.path)
states = bk.load.states()
stru = ['Hpc','BLA']
plt.figure()
for s in stru:
    t,c = synchrony(neurons[metadata.Region == s],0.025,201)
    plt.plot(t,c)
bk.plot.intervals(states['Rem'],'orange')
bk.plot.intervals(states['sws'],'grey')

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy
60  neurons


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100% |########################################################################|


94  neurons


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100% |########################################################################|


In [99]:
c

array([   nan,    nan,    nan, ...,    nan,    nan, 22679.])

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


60  neurons


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100% |########################################################################|


In [17]:
# path = bk.sessions()['Path'][4]
# bk.current_session(path)
# neurons,metadata = bk.loadSpikeData(bk.path)

methods = ['firing','coactivation','corr']
sms = [5001,5001,201]
plt.figure()
sub = plt.subplot(3,1,1)
for i,m in enumerate(methods):
    plt.subplot(len(methods),1,i+1,sharex = sub)
    syn = synchrony(neurons,metadata,0.05,['Hpc','BLA'],method = m ,sm = sms[i])
    plt.xlabel('')
    if i == 0:
        plt.title('Firing Rates')
    else:
        plt.title('Syncrhony calculated using '+ m )
plt.suptitle(bk.load.session)
plt.show()

252
0.05


TypeError: '(slice(None, None, None), 0)' is an invalid key

,Shank,Id,Region,Type
0,1,2,Hpc,Pyr
1,2,2,Hpc,Pyr
2,2,3,Hpc,Pyr
3,2,4,Hpc,Pyr
4,2,5,Hpc,Pyr
...,...,...,...,...
247,19,2,BLA,Pyr
248,19,3,BLA,Pyr
249,19,4,BLA,Pyr
250,19,5,BLA,Pyr


In [214]:
plt.subplot(3,1,1)
plt.title('Firing Rates')

Text(0.5, 1.0, 'Firing Rates')

In [184]:
methods = ['firing','coactivation','corr']
sms = [11,1001,201]
plt.figure()
sub = plt.subplot(3,1,1)
for i,m in enumerate(methods):
    plt.subplot(len(methods),1,i+1)
    syn = synchrony(neurons,metadata,100,['Hpc','BLA'],method = m ,sm = sms[i])
    plt.xlabel('')
    plt.title('Synchrony calculated using '+ m )
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  


40
100
computing Hpc
computing BLA
40
100
computing Hpc
computing BLA
40
100


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100% |########################################################################|


computing Hpc
computing BLA


In [28]:
plt.figure()
plt.subplot(1,3,1)
plt.imshow(stru_corr['Hpc'][21000,:,:])
plt.clim(-1,1)
plt.subplot(1,3,2)
plt.imshow(np.abs(stru_corr['Hpc'][21000,:,:]))
plt.clim(-1,1)
plt.subplot(1,3,3)
plt.colorbar()

NameError: name 'stru_corr' is not defined

In [10]:
for i in range(0,binned.shape[1],step):
    plt.clf()
    corr = np.corrcoef(binned[:,i:i+winSize])
    corr[np.isnan(corr)] = 0
    sns.heatmap(corr,square = True,vmin = 0,vmax = 0.1,cmap = 'viridis')
    plt.title(i)
    plt.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyboardInterrupt: 

In [255]:
a,t = binSpikes(neurons)

In [259]:
plt.plot(np.correlate(a[0],a[1],'full'))

In [126]:
g = scipy.signal.gaussian(25,1)/50

In [127]:
x = np.arange(0,100,0.1)

In [128]:
y = np.sin(x)

In [129]:
y_g = np.convolve(y,g,'same')

In [133]:
plt.plot(x,y)
plt.plot(x+5,y_g)

In [134]:
bk.load.session

'Rat08-20130713'

In [135]:
states = bk.load.states()

In [137]:
lfp = bk.load.loadLFP()

TypeError: loadLFP() missing 1 required positional argument: 'path'